# Pseudobulk analysis of differential expression

In [34]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

pandas2ri.activate()
DESeq2 = importr("DESeq2")

from rpy2.robjects import default_converter
from rpy2.robjects.conversion import rpy2py
base = importr("base")

In [35]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time, gmtime, sleep
import sys
sys.path.append("/home/jovyan/HSE-Bioinformatics")
sys.path.append("/home/jovyan/diploma_scripts/scripts")
from bio import *
import dcona_runs
import dcona
from tqdm.auto import tqdm; tqdm.pandas()
import telegram_send
import multiprocessing
import os

In [3]:
SHARED_PATH = Path('/home/jovyan/shared')
TCGA_PATH = SHARED_PATH / 'TCGA_data'
OUTPUT_PATH = SHARED_PATH / 'narek/outputs'
BRCA_DATA = SHARED_PATH / 'narek/Diplom2/data_BRCA'

TCGA_STUDY = 'TCGA-BRCA'
iso_type = "pan_cancer_exclusive_log2_FPM_DESeq2"
annotation = rt(BRCA_DATA/'annotation.tsv')


# prep expression marix and gene list

In [49]:
def deseq(meta: pd.DataFrame, counts: pd.DataFrame, formula: str, ref: str, exp: str):
    dds = DESeq2.DESeqDataSetFromMatrix(countData=counts, colData=meta, design=ro.Formula(formula))
    dds = DESeq2.DESeq(dds)
    print(f"Group_{exp}_vs_{ref}")
    resR = DESeq2.results(dds, name=f"Group_{ref}_vs_{exp}")
    res = r_to_df(resR)
    res = res.sort_values("padj")
    res = res.loc[res["padj"] < 0.05]
    res = res.loc[res["log2FoldChange"].abs() > 0.5]

    return res


def r_to_df(r_df):
    with localconverter(default_converter + pandas2ri.converter):
        return rpy2py(base.as_data_frame(r_df))

In [13]:
# gene_exp = rt(f"{TCGA_PATH}/{TCGA_STUDY}/RSEM_transcript_pan_cancer_log2_FPKM_DESeq2.tsv")
# mirna = rt(f"{TCGA_PATH}/{TCGA_STUDY}/isoMiRmap_{iso_type}.tsv")
# descr = rc(SHARED_PATH/'narek/outputs/sample_cuts/DROSHA_normal_ENST00000513349_62_samples.csv')
# raw_ts_inter = rt(SHARED_PATH/'miRNA_predictions_BRCA/TargetScan.tsv', i=None)
# data = dcona_runs.DataProcessor(gene_exp, annotation, mirna)
# data.cutoff_expressions(subtypes=cancer_subtypes)
# data.cutoff_mirnas()
# brca_ts_filtered, descr_ts_filtered, interaction_ts_filtered = data.final_data(descr, raw_ts_inter)
# data.expr_united.to_csv('~/diploma_scripts/aux/expr95_mirna99.csv', columns=[])

In [5]:
gene_ids = rc('~/diploma_scripts/aux/expr95_mirna99.csv').index
gene_ids

Index(['ENST00000361624', 'ENST00000362079', 'ENST00000361381',
       'ENST00000361899', 'ENST00000331523', 'ENST00000361739',
       'ENST00000361453', 'ENST00000361390', 'ENST00000361789',
       'ENST00000387347',
       ...
       'hsa-miR-92b-3p|0', 'hsa-miR-10a-5p|-1', 'hsa-miR-93-5p|+3',
       'hsa-miR-99b-5p|+1', 'hsa-miR-128-3p|0', 'hsa-miR-15a-5p|0',
       'hsa-miR-361-3p|0', 'hsa-miR-629-5p|0', 'hsa-miR-143-3p|+3',
       'hsa-let-7b-3p|0'],
      dtype='object', length=23518)

In [6]:
gene_exp_counts = rt(f"{TCGA_PATH}/{TCGA_STUDY}/RSEM_transcript_pan_cancer_counts.tsv")
mirna_counts = rt(f"{TCGA_PATH}/{TCGA_STUDY}/isoMiRmap_pan_cancer_exclusive_counts.tsv")

In [11]:
mirna_counts_5prime = dcona_runs.isomir_groupby_5prime(mirna_counts, counts_data=True)

In [24]:
united_data = pd.concat(tcga_match_samples(
    dcona_runs.remove_transcript_version(gene_exp_counts.drop('gene symbol', axis=1)),
    mirna_counts_5prime.drop('median', axis=1))
                       ).loc[gene_ids]

In [154]:
descr = rc(SHARED_PATH/'narek/outputs/sample_cuts/AGO2_luminalA_ENST00000220592_113_samples.csv')
descr = descr.set_index('Sample')

In [155]:
extracted_united = dcona_runs.extract_samples(united_data, descr.index)

In [10]:
from importlib import reload
reload(dcona_runs)

<module 'dcona_runs' from '/home/jovyan/diploma_scripts/scripts/dcona_runs.py'>

In [ ]:
res = deseq(meta=descr, counts=extracted_united, formula="~Group", ref="Low_expr", exp="High_expr")
res

In [156]:
extracted_united.loc['hsa-miR-148a-3p|0', descr.loc[descr['Group'] == 'Low_expr'].index].mean()

296971.7894736842

In [157]:
extracted_united.loc['hsa-miR-148a-3p|0', descr.loc[descr['Group'] == 'High_expr'].index].mean()

141277.4642857143

# Results

In [136]:
mir_dict = dict()
for i in range(1, 11):
    df = rt(OUTPUT_PATH/'experiments'/str(i)/'DESeq2_all_mirnas.tsv')
    df = df.loc[df.index.str.contains('hsa')]
    mir_dict[i] = df
    del df

In [167]:
np.unique(np.sign(mir_dict[10]['log2FoldChange']), return_counts=True)

(array([-1.,  1.]), array([285, 152]))

In [164]:
mir_dict[7]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
hsa-miR-577|0,9.092564,2.525417,0.549824,4.593138,0.000004,0.005309
